In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
def ner_by_nltk(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    # вытаскиваем только имена собственные
    return [token for token, tag in tagged_tokens if tag in ('NNP', 'NNPS')]

def ner_by_spacy(text):
    doc = nlp(text)
    return [X.text for X in doc.ents]

In [3]:
def top_20(results):
    ner_array, counts_array = np.unique(results, return_counts=True)
    
    index = pd.Series(ner_array, name='NER')
    top = pd.DataFrame(counts_array, index=index, columns=['counts'])
    top.sort_values(by=['counts'], ascending=False, inplace=True)
    
    return top.head(20)

In [4]:
import pickle

with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)

In [5]:
combine_df.head(n=2)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."


In [6]:
import nltk
from nltk.tokenize import word_tokenize

In [7]:
#nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

In [8]:
nltk_results = []
for tweet in tqdm(combine_df['tweet']):
    result = ner_by_nltk(tweet)
    if result:
        nltk_results.extend(result)

100%|███████████████████████████████████| 49159/49159 [01:38<00:00, 499.92it/s]


In [9]:
top_20(nltk_results)

,counts
NER,
xxx,57
xoxo,26
xenophobia,16
youtuber,14
xx,14
kinky,9
democrats,6
october,5
youtube,4


In [10]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [11]:
spacy_results = []
for tweet in tqdm(combine_df['tweet']):
    result = ner_by_spacy(tweet)
    if result:
        spacy_results.extend(result)

100%|████████████████████████████████████| 49159/49159 [13:19<00:00, 61.48it/s]


In [12]:
top_20(spacy_results)

,counts
NER,
today,1552
friday,675
tomorrow,611
first,560
one,512
summer,501
sunday,495
tonight,454
orlando,385


**Разница работы:**

In [21]:
sentence = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices trump wednesday'

In [22]:
sentence_tokens = nltk.word_tokenize(sentence)
nltk.pos_tag(sentence_tokens)

[('European', 'JJ'),
 ('authorities', 'NNS'),
 ('fined', 'VBD'),
 ('Google', 'NNP'),
 ('a', 'DT'),
 ('record', 'NN'),
 ('$', '$'),
 ('5.1', 'CD'),
 ('billion', 'CD'),
 ('on', 'IN'),
 ('Wednesday', 'NNP'),
 ('for', 'IN'),
 ('abusing', 'VBG'),
 ('its', 'PRP$'),
 ('power', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('mobile', 'JJ'),
 ('phone', 'NN'),
 ('market', 'NN'),
 ('and', 'CC'),
 ('ordered', 'VBD'),
 ('the', 'DT'),
 ('company', 'NN'),
 ('to', 'TO'),
 ('alter', 'VB'),
 ('its', 'PRP$'),
 ('practices', 'NNS'),
 ('trump', 'VBP'),
 ('wednesday', 'NN')]

In [26]:
doc = nlp(sentence)
[(X.text, X.label_) for X in doc.ents]

[('European', 'NORP'),
 ('Google', 'ORG'),
 ('$5.1 billion', 'MONEY'),
 ('Wednesday', 'DATE'),
 ('wednesday', 'DATE')]